In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers 

tablename = "factFOIFlowRequestStatusDetails"
runcycleid = etl_helpers.start_run_cycle(tablename)

os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:
    df_lastrun = spark.sql(f"SELECT runcyclestartat as createddate FROM dimruncycle WHERE packagename = \"{tablename}\" AND success = 't' ORDER BY runcycleid DESC LIMIT 1")
    
    if df_lastrun.count() > 0:
        lastruntime = df_lastrun.first().createddate.strftime("%Y-%m-%d %H:%M:%S")
    else:
        lastruntime = "2019-01-01 00:00:00"
    print(lastruntime)

    query = f"""
        WITH assignees AS (
        SELECT 
            username,
            CASE 
            WHEN firstname IS NOT NULL THEN CONCAT(firstname, ' ', lastname)
            ELSE username
            END AS fullname
        FROM foi_mod.FOIAssignees
        )

        SELECT
            FMR.foiministryrequestid AS foiministryrequestid,
            FMR.version AS requestversion,
            FMR.isactive AS requestisactive,
            FMR.axisrequestid AS filerequestnumber,
            PA.iaocode AS ministrycode,
            FMR.startdate AS requeststartdate,
            FMR.duedate AS requestduedate,
            FMR.assignedto AS iaoassignedto,
            FMR.created_at AS requestcreatedat,
            FMR.updated_at AS requestupdatedat,
            FMR.createdby AS requestcreatedby,
            FMR.updatedby AS requestupdatedby,
            FMR.requeststatusid AS ministryrequeststatusid,
            FMR.assignedgroup AS iaoassignedgroup,
            FMR.assignedministryperson AS ministryperson,
            FMR.assignedministrygroup AS ministryteam,
            FMR.cfrduedate AS requestcfrduedate,
            FMR.closedate AS requestclosedate,
            a1.fullname AS assigneefullname,
            a2.fullname AS iaoassigneefullname,
            try_cast(FMR.recordspagecount AS STRING) AS recordspagecount,
            try_cast(FMR.axislanpagecount AS STRING) AS lanpagecount
        FROM foi_mod.FOIMinistryRequests FMR
        INNER JOIN foi_mod.ProgramAreas PA 
        ON FMR.programareaid = PA.programareaid

        LEFT JOIN assignees a1 
        ON a1.username = FMR.assignedministryperson

        LEFT JOIN assignees a2 
        ON a2.username = FMR.assignedto

        WHERE FMR.created_at > '{lastruntime}'
        """

    print(query)

    df = spark.sql(query)
    df.show()

    if df.count() == 0:
        raise Exception("no changes for today")


    # order of columns here is important!
    df_mapped = df.selectExpr(
            f"{runcycleid} as runcycleid",
            "filerequestnumber AS foirequestnumber",
            "requestversion AS version",
            "requestisactive AS isactive",
            "ministryrequeststatusid AS requestatusid",
            "ministrycode AS ministrycode",
            "iaoassignedgroup AS assignedgroup",
            "ministryperson AS assignedministryperson",
            "ministryteam AS assignedministrygroup",
            "requeststartdate AS startdate",
            "requestduedate AS duedate",
            "requestcfrduedate AS cfrduedate",
            "requestclosedate AS closedate",
            "requestcreatedby AS createdby",
            "requestcreatedat AS createddate",
            "requestupdatedby AS modifiedby",
            "requestupdatedat AS modifieddate",
            "iaoassignedto AS iaoassignedto",
            "iaoassigneefullname AS iaoassigneefullname",
            "assigneefullname AS ministryassigneefullname",
            "recordspagecount AS recordspagecount",
            "lanpagecount AS lanpagecount",            
        )
    df_mapped.show()
    etl_helpers.append_with_incrementing_id(df_mapped, table_name=tablename, id_column="foiflowrequeststatusdetailsid", database="hive_metastore.default")
    etl_helpers.end_run_cycle(runcycleid, 't', tablename)
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', tablename, "Credentials not available")
    raise Exception("notebook failed") from e
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', tablename)
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', tablename, f"An error occurred: {e}")
        raise Exception("notebook failed") from e